# KorNLI 분류

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


## 데이터셋 로드 및 구조 확인

In [ ]:
from datasets import load_dataset

# NLI 데이터 중 학습 데이터(split="train")를 로드.
cs = load_dataset("klue", "nli", split="train")

# NLI 데이터 중 테스트 데이터(split="validation")을 로드.
test_cs = load_dataset("klue", "nli", split="validation")

# 학습 데이터를 로드하여 9:1 비율로 분할하고 각각 train_cs와 valid_cs에 저장
cs = cs.train_test_split(0.1, seed=777)
train_cs = cs["train"]
valid_cs = cs["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
train_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 22498
})

In [ ]:
valid_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 2500
})

In [ ]:
test_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 3000
})

## 데이터셋 전처리

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda train_cs: '[CLS] ' + str(train_cs['premise']) + ' [SEP] ' + str(train_cs['hypothesis']) + ' [SEP]', train_cs))
validation_sentences = list(map(lambda valid_cs: '[CLS] ' + str(valid_cs['premise']) + ' [SEP] ' + str(valid_cs['hypothesis']) + ' [SEP]', valid_cs))
test_sentences = list(map(lambda test_cs: '[CLS] ' + str(test_cs['premise']) + ' [SEP] ' + str(test_cs['hypothesis']) + ' [SEP]', test_cs))

In [ ]:
train_labels = train_cs['label']
validation_labels = valid_cs['label']
test_labels = test_cs['label']

In [ ]:
validation_sentences[:5]

['[CLS] 우리는 마당에서 물놀이도 할 수 있었다 [SEP] 마당이 넓고 좋았다. [SEP]',
 '[CLS] 미국의 상원법사 위원회는 코미 전 국장에게 러시아 대선 개입 관련 메모를 제출하도록 요구하였고, 백악관도 관련 녹취기록을 요구하였다. [SEP] 미국의 상원법사 위원회는 백악관에 녹취기록 요구는 하지 않았다. [SEP]',
 '[CLS] 10월 29일 기준, 전체 163개국의 회원국 중 104개국이 오콘조이웨알라 후보를 지지하는 것으로 알려졌다. [SEP] 10월 29일 기준 모든 회원국이 오콘조이웨알라 후보를 지지하는 것으로 알려졌다. [SEP]',
 '[CLS] 영화보는내내 전개와 상황이 지루해서 몰입도 안됬습니다. [SEP] 전개와 상황이 지루해서 영화 보는 내내 몰입이 안됐습니다. [SEP]',
 '[CLS] 청소년들이 직접 건의해 잘못된 표기를 시정하게 했고, 정부만의 역할이 아닌 대한민국 국민 누구나 민간외교관이 될 수 있다는 것을 보여줬습니다. [SEP] 청소년들의 활동으로 인해 많은 사람들이 민간외교관에 관심을 가지게 되었습니다. [SEP]']

In [ ]:
validation_labels[:5]

[1, 2, 2, 0, 1]

## BERT 토크나이저를 이용한 전처리

In [ ]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
tokenized_text = tokenizer.tokenize('안녕하세요. 자연어 처리를 배울거에요.')
input_id = tokenizer.convert_tokens_to_ids(tokenized_text)

print('토큰화 된 문장 :', tokenized_text)
print('정수 인코딩 된 문장 :', input_id)

토큰화 된 문장 : ['안녕', '##하', '##세요', '.', '자연', '##어', '처리', '##를', '배울', '##거', '##에', '##요', '.']
정수 인코딩 된 문장 : [5891, 2205, 5971, 18, 3941, 2051, 4211, 2138, 9402, 2180, 2170, 2182, 18]


In [ ]:
# 최대 길이는 128
max_len = 128

def data_to_tensor (sentences, labels, max_len):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")

  # 실제 토큰 위치에는 1, 패딩 토큰 위치에는 0을 넣은 리스트인 어텐션 마스크 제작.
  # 정수 인코딩 결과가 [231, 52, 45, 0, 0, 0]이 있다면 231, 52, 45는 실제 토큰이고 0은 패딩 토큰이므로
  # 어텐션 마스크는 [1, 1, 1, 0, 0, 0]
  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [ ]:
# 학습 데이터, 검증 데이터, 테스트 데이터에 대해서
# 정수 인코딩 결과, 레이블, 어텐션 마스크를 각각 inputs, labels, masks에 저장.
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)

In [ ]:
tokenizer.decode([2])

'[CLS]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

In [ ]:
print('정수 인코딩 결과:', test_inputs[0])
print('-' * 20)
print('원본 문장 복원 결과:', tokenizer.decode(test_inputs[0]))
print('-' * 20)
print('어텐션 마스크:', test_masks[0])
print('-' * 20)
print('샘플의 길이:', len(test_inputs[0]))
print('-' * 20)
print('레이블:', test_labels[0])

정수 인코딩 결과: tensor([    2, 25313,  2377,  2031,  2073, 20812,  2116,  1513,  2259,  1129,
        24094, 20812, 27135,  9753,  2052,  3662, 11800,    18,     3,  3711,
         1129, 27135,  2119,  9753,  2073,  5040,  3598,  3606,    18,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    

## 데이터 배치화

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 22498
검증 데이터의 크기: 2500
테스트 데이터의 크기: 3000


## GPU가 정상 셋팅되었는지 확인.  
Colab에서 GPU를 사용하기 위해서는 아래와 같이 설정이 되어있어야만 합니다.  

* 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > 'GPU' 선택

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 모델 로드하기

In [ ]:
num_labels = 3

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## 모델 학습

In [ ]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 3

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def metrics(predictions, labels):
    # predictions: 모델이 예측한 결과값들의 리스트 또는 배열
    # labels: 실제 정답 레이블들의 리스트 또는 배열

    # 예측값과 실제 레이블을 별도의 변수에 할당
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 계산

    # 정확도 (Accuracy)
    # 전체 예측 중에서 올바르게 예측한 비율
    accuracy = accuracy_score(y_true, y_pred)

    # 매크로 평균 F1 점수 (Macro-averaged F1 Score)
    # 클래스별로 F1 점수를 계산한 후, 그 평균을 구함
    # zero_division=0 옵션은 분모가 0일 경우 0을 반환하도록 설정
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)

    # 마이크로 평균 F1 점수 (Micro-averaged F1 Score)
    # 전체 데이터에 대해 단일 F1 점수를 계산
    # 클래스 불균형이 심한 경우에 적합
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)

    # 가중 평균 F1 점수 (Weighted-averaged F1 Score)
    # 각 클래스의 F1 점수에 해당 클래스의 샘플 수를 가중치로 곱한 후 평균을 구함
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 계산된 메트릭 결과를 딕셔너리 형태로 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}
    return metrics

In [ ]:
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """

    total_train_loss = 0  # 학습 손실을 누적할 변수 초기화
    model.train()  # 모델을 학습 모드로 설정

    # 학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader), desc="Training Batch"):
        batch = tuple(t.to(device) for t in batch)  # DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        # 모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 손실값 추출
        loss = outputs.loss

        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()  # 역전파를 통해 그래디언트 계산
        optimizer.step()  # 매개변수 업데이트

        total_train_loss += loss.item()  # 총 손실에 더함

    avg_train_loss = total_train_loss / len(train_dataloader)  # 평균 학습 손실 계산

    return avg_train_loss  # 평균 학습 손실 반환

In [ ]:
def evaluate(model, validation_dataloader, device):
    """
    모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수입니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체.
    validation_dataloader (torch.utils.data.DataLoader): 검증 데이터셋의 DataLoader.
    device (torch.device): 평가에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 검증 손실값.
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전.
    """

    model.eval()  # 모델을 평가 모드로 설정

    total_eval_loss = 0  # 검증 손실을 누적할 변수 초기화
    predictions, true_labels = [], []  # 예측값과 실제 라벨값을 저장할 리스트 초기화

    # 검증 데이터로더를 순회하며 배치 단위로 평가
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)  # 배치 데이터를 디바이스로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        with torch.no_grad():  # 그래디언트 계산을 수행하지 않음
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 모델 출력에서 손실값 추출 (수정된 부분)
        if outputs.loss is not None:
            loss = outputs.loss
            total_eval_loss += loss.item()  # 총 손실에 더함

        logits = outputs.logits.detach().cpu().numpy()  # 모델 예측값(로짓)을 numpy 배열로 변환
        label_ids = b_labels.to('cpu').numpy()  # 실제 라벨값을 numpy 배열로 변환

        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 (예시: logits = [3.513, -0.309, -2.111] ==> 예측: 0)
        predictions.extend(np.argmax(logits, axis=1).flatten()) # 예측된 클래스를 리스트에 추가
        true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가

    eval_metrics = metrics(predictions, true_labels)

    return total_eval_loss / len(validation_dataloader), eval_metrics

In [ ]:
# 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루프
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # 학습 단계
    train_epoch(model, train_dataloader, optimizer, device)

    print("\nRunning Validation...")
    # 검증 단계
    avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
    print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
    print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
    print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

    # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
    if avg_val_loss < min_val_loss:
        print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
        # 베스트 모델 저장
        torch.save(model.state_dict(), 'model_checkpoint.pt')
        # 최소 검증 손실 업데이트
        min_val_loss = avg_val_loss

======== Epoch 1 / 3 ========


Training Batch: 704it [01:51,  6.33it/s]



Running Validation...
  Validation Loss: 0.47
  Accuracy: 0.82
  F1 Macro: 0.82
  F1 Micro: 0.82
  F1 Weighted: 0.82
Validation loss decreased (inf --> 0.47).  Saving model ...
======== Epoch 2 / 3 ========


Training Batch: 704it [01:51,  6.33it/s]



Running Validation...
  Validation Loss: 0.44
  Accuracy: 0.84
  F1 Macro: 0.84
  F1 Micro: 0.84
  F1 Weighted: 0.84
Validation loss decreased (0.47 --> 0.44).  Saving model ...
======== Epoch 3 / 3 ========


Training Batch: 704it [01:51,  6.34it/s]



Running Validation...
  Validation Loss: 0.54
  Accuracy: 0.82
  F1 Macro: 0.82
  F1 Micro: 0.82
  F1 Weighted: 0.82


## 모델 저장과 로드

In [ ]:
%pwd

'/content'

In [ ]:
%ls -al

total 432208
drwxr-xr-x 1 root root      4096 Feb 11 16:47 ./
drwxr-xr-x 1 root root      4096 Feb 11 16:41 ../
drwxr-xr-x 4 root root      4096 Feb  8 14:20 .config/
-rw-r--r-- 1 root root 442562069 Feb 11 17:03 model_checkpoint.pt
drwxr-xr-x 1 root root      4096 Feb  8 14:21 sample_data/


In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

<All keys matched successfully>

# 테스트 데이터에 대한 평가

In [ ]:
avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

  Test Loss: 0.52
  Accuracy: 0.79
  F1 Macro: 0.79
  F1 Micro: 0.79
  F1 Weighted: 0.79


# 예측

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='softmax')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
inputs = {"text" : "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.", "text_pair" : "어떤 방에서도 흡연은 금지됩니다."}

In [ ]:
result = pipe([inputs])
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.012474408373236656}, {'label': 'LABEL_1', 'score': 0.05696354806423187}, {'label': 'LABEL_2', 'score': 0.9305620193481445}]]


return_all_scores를 제거하면 정답으로 확신하는 레이블만 리턴합니다.

In [ ]:
# return_all_scores 제거
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, function_to_apply='softmax')

In [ ]:
result = pipe([inputs])
print(result)

[{'label': 'LABEL_2', 'score': 0.9305620193481445}]


In [ ]:
label_dict = {'LABEL_0' : '얽힘', 'LABEL_1' : '중립', 'LABEL_2' : '모순'}

In [ ]:
def prediction(sent1, sent2):
  text = {"text" : sent1, "text_pair" : sent2}
  result = pipe(text)
  return [label_dict[result['label']]]

In [ ]:
sent1 = "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다."
sent2 = "어떤 방에서도 흡연은 금지됩니다."

In [ ]:
prediction(sent1, sent2)

['모순']

In [ ]:
sent1 = "저는, 그냥 알아내려고 거기 있었어요."
sent2 = "나는 돈이 어디로 갔는지 이해하려고 했어요."

In [ ]:
prediction(sent1, sent2)

['중립']

In [ ]:
sent1 = "저는 그것을 이해하려고 거기 있었어요."
sent2 = "저는 이해하려고 노력하고 있었어요."

In [ ]:
prediction(sent1, sent2)

['얽힘']